# 🕸️ 第一章 练习 2: 因果图与 DAG

---

## 从「看图说话」到「看图断因果」

在上一个练习中，我们学习了潜在结果框架。但是，当你遇到这样的问题时：

> "这个变量应该控制吗？控制了会怎样？不控制又会怎样？"

单纯靠数学公式就不够直观了。这时候，**因果图 (Causal Graph)** 就派上用场了！

### 一个有趣的例子 🏥

你是一名流行病学家，在研究「运动」对「健康」的影响。你发现：

- 经常运动的人更健康 ✓
- 但经常运动的人收入也更高...
- 而高收入的人本来就更健康（更好的医疗、饮食）...

所以运动到底有没有用？收入是怎么回事？应该怎么分析？

用因果图，这个问题一画就清楚了！

---

## 📚 本节学习目标

1. 理解因果图的基本概念
2. 识别三种核心结构：混淆、中介、碰撞
3. 理解后门路径和 d-分离
4. 掌握「应该控制哪些变量」的决策方法

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Set, Tuple

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！")

---

## 🌟 Part 1: 什么是因果图 (DAG)？

### DAG = Directed Acyclic Graph（有向无环图）

- **有向 (Directed)**: 箭头指向因果的方向（原因 → 结果）
- **无环 (Acyclic)**: 不能有循环（A → B → C → A 是不允许的）
- **图 (Graph)**: 用节点和边表示变量之间的关系

### 最简单的因果图

```
T → Y
```

这表示：T **导致** Y（T 是原因，Y 是结果）

### 经典的混淆结构

```
    X
   ↙ ↘
  T   Y
   ↘ ↗
```

等价于：
- X → T（X 影响 T）
- X → Y（X 影响 Y）  
- T → Y（T 影响 Y）

这里 X 就是**混淆变量 (Confounder)**！

### 🎨 用代码画因果图

让我们用一个简单的函数来可视化因果图：

In [ ]:
def draw_dag(edges, title="因果图 (DAG)", highlight_path=None):
    """
    画一个简单的因果图
    
    Args:
        edges: 边的列表，如 [("X", "T"), ("X", "Y"), ("T", "Y")]
        title: 图的标题
        highlight_path: 要高亮的路径
    """
    try:
        import networkx as nx
        
        G = nx.DiGraph()
        G.add_edges_from(edges)
        
        # 设置节点位置
        pos = nx.spring_layout(G, seed=42)
        
        plt.figure(figsize=(8, 6))
        
        # 画边
        nx.draw_networkx_edges(G, pos, edge_color='gray', 
                               arrows=True, arrowsize=20,
                               connectionstyle="arc3,rad=0.1")
        
        # 如果有高亮路径
        if highlight_path:
            highlight_edges = [(highlight_path[i], highlight_path[i+1]) 
                              for i in range(len(highlight_path)-1)]
            nx.draw_networkx_edges(G, pos, edgelist=highlight_edges,
                                   edge_color='red', width=2,
                                   arrows=True, arrowsize=25)
        
        # 画节点
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                               node_size=2000, alpha=0.9)
        nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')
        
        plt.title(title, fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("需要安装 networkx: pip install networkx")
        print(f"\n边: {edges}")

# 画一个简单的混淆结构
confounding_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]
draw_dag(confounding_edges, "经典混淆结构: X 是混淆变量")

---

## 🔍 Part 2: 三种核心因果结构

因果图中有三种最基本的结构，理解它们是掌握因果推断的关键！

### 结构 1: 叉子 (Fork) - 混淆

```
T ← X → Y
```

**特点**: X 同时影响 T 和 Y

**例子**: 
- X = 年龄
- T = 是否喝咖啡
- Y = 是否有心脏病

年轻人喝咖啡更多，但年轻人心脏病也更少。如果不控制年龄，会错误地认为咖啡预防心脏病！

**规则**: 需要控制 X 来阻断混淆

---

### 结构 2: 链条 (Chain) - 中介

```
T → M → Y
```

**特点**: T 通过 M 影响 Y

**例子**:
- T = 受教育程度
- M = 收入
- Y = 健康状况

教育通过提高收入来改善健康。M 是中介变量。

**规则**: 
- 如果想估计 T 的**总效应**，不要控制 M
- 如果想估计 T 的**直接效应**，需要控制 M

---

### 结构 3: 碰撞 (Collider) - 碰撞变量

```
T → C ← Y
```

**特点**: T 和 Y 都影响 C

**例子** (经典！):
- T = 演技好
- Y = 颜值高
- C = 成为明星

成为明星需要演技**或**颜值（至少一个）。在明星群体中，演技好和颜值高会呈现负相关——因为只靠颜值的人演技可能不好，只靠演技的人可能不是靠脸吃饭。

**规则**: 千万不要控制碰撞变量 C！否则会创造虚假关联

In [ ]:
# 画出三种结构
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

structures = [
    ([("X", "T"), ("X", "Y")], "Fork (叉子/混淆)\nT ← X → Y\n控制 X ✓"),
    ([("T", "M"), ("M", "Y")], "Chain (链条/中介)\nT → M → Y\n通常不控制 M"),
    ([("T", "C"), ("Y", "C")], "Collider (碰撞)\nT → C ← Y\n绝对不控制 C ✗")
]

for idx, (edges, title) in enumerate(structures):
    ax = axes[idx]
    try:
        import networkx as nx
        G = nx.DiGraph()
        G.add_edges_from(edges)
        pos = nx.spring_layout(G, seed=42)
        
        nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray', 
                               arrows=True, arrowsize=20)
        nx.draw_networkx_nodes(G, pos, ax=ax, node_color='lightblue', 
                               node_size=1500, alpha=0.9)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=12, font_weight='bold')
        ax.set_title(title, fontsize=11)
        ax.axis('off')
    except:
        ax.text(0.5, 0.5, f"边: {edges}\n{title}", ha='center', va='center')
        ax.axis('off')

plt.tight_layout()
plt.show()

### 🧪 练习: 识别因果结构

In [ ]:
def identify_structure(edges: List[Tuple[str, str]], node: str) -> str:
    """
    根据 DAG 边，识别给定节点的因果结构类型
    
    结构类型:
    - "confounder": 混淆变量 (出度 >= 2，像叉子的中心)
    - "mediator": 中介变量 (入度 >= 1 且 出度 >= 1，在链条中间)
    - "collider": 碰撞变量 (入度 >= 2，像碰撞的中心)
    
    Args:
        edges: 边列表，如 [("X", "T"), ("X", "Y"), ("T", "Y")]
        node: 要识别的节点
    
    Returns:
        结构类型字符串
    """
    # 计算入度：有多少条边指向这个节点
    in_degree = sum(1 for edge in edges if edge[1] == node)
    
    # 计算出度：这个节点有多少条边指出去
    out_degree = sum(1 for edge in edges if edge[0] == node)
    
    # TODO: 根据入度和出度判断结构类型
    # 碰撞变量 (collider): 入度 >= 2, 出度 = 0（多个箭头指向它）
    # 混淆变量 (confounder): 出度 >= 2（它指向多个节点）
    # 中介变量 (mediator): 入度 >= 1 且 出度 >= 1（在链条中间）
    
    # 👈 你的代码
    if in_degree >= 2 and out_degree == 0:
        return None  # 这是什么类型？
    elif out_degree >= 2:
        return None  # 这是什么类型？
    elif in_degree >= 1 and out_degree >= 1:
        return None  # 这是什么类型？
    else:
        return "other"

In [ ]:
# 测试你的代码
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n识别每个节点的角色:")

for node in ["X", "T", "Y"]:
    structure = identify_structure(test_edges, node)
    if structure:
        print(f"   节点 {node}: {structure}")
    else:
        print(f"   节点 {node}: [未完成]")

---

## 🛤️ Part 3: 路径与后门准则

### 什么是「路径」？

在因果图中，从 T 到 Y 可能有多条路径：

```
    X
   ↙ ↘
  T → Y
```

从 T 到 Y 有两条路径：
1. **直接路径 (因果路径)**: T → Y ✅
2. **后门路径**: T ← X → Y ⚠️

### 什么是「后门路径」？

后门路径是从 T 到 Y 的路径中，第一个箭头**指向 T** 的路径。

- T → Y 不是后门（箭头从 T 出发）
- T ← X → Y 是后门（第一个箭头指向 T）

**后门路径会造成混淆偏差！我们需要把它阻断。**

### 后门准则 (Backdoor Criterion)

要无偏地估计 T 对 Y 的因果效应，需要找到一组变量 Z 满足：

1. Z 阻断了所有从 T 到 Y 的后门路径
2. Z 不包含 T 的任何后代

In [ ]:
def find_all_paths(edges: List[Tuple[str, str]], 
                   start: str, 
                   end: str,
                   ignore_direction: bool = True) -> List[List[str]]:
    """
    找出 DAG 中从 start 到 end 的所有路径
    
    Args:
        edges: 边列表
        start: 起始节点
        end: 终止节点
        ignore_direction: 是否忽略边的方向（找后门路径时需要忽略）
    
    Returns:
        路径列表，每条路径是节点序列
    """
    # 构建邻接表
    adj = {}
    for u, v in edges:
        if u not in adj:
            adj[u] = []
        adj[u].append(v)
        
        if ignore_direction:  # 如果忽略方向，也添加反向边
            if v not in adj:
                adj[v] = []
            adj[v].append(u)
    
    paths = []
    
    # TODO: 使用 DFS 找出所有路径
    def dfs(current, path, visited):
        """
        深度优先搜索找路径
        
        current: 当前节点
        path: 当前路径
        visited: 已访问节点集合
        """
        # 如果到达终点，保存路径
        if current == end:
            paths.append(path.copy())
            return
        
        # 遍历邻居
        for neighbor in adj.get(current, []):
            if neighbor not in visited:
                # 👈 你的代码：将邻居加入已访问，递归调用，然后移除
                # 提示: 
                # 1. visited.add(neighbor)
                # 2. path.append(neighbor)
                # 3. dfs(neighbor, path, visited)
                # 4. path.pop()
                # 5. visited.remove(neighbor)
                pass
    
    dfs(start, [start], {start})
    return paths

In [ ]:
def identify_backdoor_paths(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str) -> List[List[str]]:
    """
    识别从 treatment 到 outcome 的后门路径
    
    后门路径: 从 T 到 Y 的路径，其中第一条边指向 T
    
    Returns:
        后门路径列表
    """
    all_paths = find_all_paths(edges, treatment, outcome, ignore_direction=True)
    
    backdoor_paths = []
    for path in all_paths:
        if len(path) < 2:
            continue
        
        # TODO: 检查是否是后门路径
        # 后门路径的第一条边指向 treatment
        # 即检查 (path[1], treatment) 是否在 edges 中
        
        first_step = path[1]  # 路径上的第二个节点
        
        # 检查边 first_step → treatment 是否存在
        # 👈 你的代码
        is_backdoor = None  # (first_step, treatment) in edges
        
        if is_backdoor:
            backdoor_paths.append(path)
    
    return backdoor_paths

In [ ]:
# 测试后门路径识别
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n寻找从 T 到 Y 的路径...")

all_paths = find_all_paths(test_edges, "T", "Y", ignore_direction=True)
print(f"\n所有路径: {all_paths}")

backdoor = identify_backdoor_paths(test_edges, "T", "Y")
print(f"后门路径: {backdoor}")

if backdoor:
    print(f"\n✅ 需要控制的变量: {set(sum(backdoor, [])) - {'T', 'Y'}}")

---

## 🔒 Part 4: 阻断路径

### 阻断规则

| 结构 | 路径 | 控制中间节点后 |
|-----|------|-------------|
| Fork | A ← X → B | **阻断** |
| Chain | A → X → B | **阻断** |
| Collider | A → X ← B | **打开**（原本是阻断的！）|

⚠️ **关键点**: 碰撞变量默认是阻断的，控制它反而会打开路径！

In [ ]:
def is_valid_adjustment_set(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str,
                            adjustment_set: Set[str]) -> bool:
    """
    检查给定的调整集是否有效（满足后门准则）
    
    后门准则要求调整集:
    1. 阻断所有后门路径
    2. 不包含 treatment 的后代
    
    Returns:
        True 如果调整集有效
    """
    # 步骤 1: 找出 treatment 的所有后代
    def find_descendants(node):
        """找出一个节点的所有后代"""
        descendants = set()
        to_visit = [n for (s, n) in edges if s == node]
        while to_visit:
            current = to_visit.pop()
            if current not in descendants:
                descendants.add(current)
                to_visit.extend([n for (s, n) in edges if s == current])
        return descendants
    
    descendants_of_T = find_descendants(treatment)
    
    # TODO: 检查调整集是否包含 treatment 的后代
    # 👈 你的代码
    contains_descendant = None  # bool(adjustment_set & descendants_of_T)
    
    if contains_descendant:
        return False  # 违反条件 2
    
    # 步骤 2: 找出所有后门路径
    backdoor_paths = identify_backdoor_paths(edges, treatment, outcome)
    
    # TODO: 检查调整集是否阻断所有后门路径
    # 一条路径被阻断当且仅当调整集包含路径上的至少一个节点（除了 T 和 Y）
    for path in backdoor_paths:
        path_nodes = set(path) - {treatment, outcome}
        # 👈 你的代码：检查 adjustment_set 是否与 path_nodes 有交集
        is_blocked = None
        
        if not is_blocked:
            return False  # 有后门路径没被阻断
    
    return True

In [ ]:
# 测试调整集
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n检验不同的调整集:")

adjustment_sets = [set(), {"X"}, {"Y"}]
for adj_set in adjustment_sets:
    result = is_valid_adjustment_set(test_edges, "T", "Y", adj_set)
    status = "✅ 有效" if result else "❌ 无效"
    print(f"   调整集 {adj_set if adj_set else '{}'}: {status}")

---

## 🎮 Part 5: 模拟验证

让我们用模拟数据来验证因果图的理论！

In [ ]:
def simulate_confounding_dag(n: int = 2000, seed: int = 42):
    """
    模拟经典混淆 DAG 的数据
    
    DAG: X → T, X → Y, T → Y
    
    数据生成过程:
    - X ~ N(0, 1)
    - T = 1 if X + noise > 0 else 0
    - Y = 1 + 2*T + 1.5*X + noise
    
    真实 ATE = 2
    """
    np.random.seed(seed)
    
    # TODO: 生成混淆变量 X
    X = None  # 👈 你的代码: np.random.randn(n)
    
    # TODO: 生成处理 T（受 X 影响）
    # T = 1 当 X + noise > 0
    T = None  # 👈 你的代码: (X + np.random.randn(n) > 0).astype(int)
    
    # TODO: 生成结果 Y（受 T 和 X 影响）
    # Y = 1 + 2*T + 1.5*X + noise
    Y = None  # 👈 你的代码
    
    df = pd.DataFrame({'X': X, 'T': T, 'Y': Y})
    
    return df

In [ ]:
def estimate_ate_with_adjustment(df: pd.DataFrame, 
                                  adjustment_vars: List[str]) -> float:
    """
    使用线性回归进行调整估计
    
    Y = b0 + b1*T + b2*X1 + b3*X2 + ...
    b1 就是调整后的 ATE 估计
    """
    from sklearn.linear_model import LinearRegression
    
    # 构建特征矩阵
    features = ['T'] + adjustment_vars
    X = df[features].values
    y = df['Y'].values
    
    # 拟合线性回归
    model = LinearRegression()
    model.fit(X, y)
    
    # T 的系数就是调整后的 ATE
    return model.coef_[0]

In [ ]:
# 运行模拟
df = simulate_confounding_dag(n=5000)

if df is not None and df['X'] is not None:
    true_ate = 2.0
    
    # 朴素估计（不控制任何变量）
    naive_ate = df[df['T']==1]['Y'].mean() - df[df['T']==0]['Y'].mean()
    
    # 调整估计（控制 X）
    adjusted_ate = estimate_ate_with_adjustment(df, ['X'])
    
    print("🔬 混淆 DAG 模拟结果:")
    print("=" * 50)
    print(f"真实 ATE: {true_ate:.4f}")
    print(f"\n朴素估计（不控制 X）: {naive_ate:.4f}")
    print(f"   偏差: {naive_ate - true_ate:+.4f} {'⚠️ 有偏！' if abs(naive_ate - true_ate) > 0.1 else ''}")
    print(f"\n调整估计（控制 X）: {adjusted_ate:.4f}")
    print(f"   偏差: {adjusted_ate - true_ate:+.4f} {'✅ 很准！' if abs(adjusted_ate - true_ate) < 0.1 else ''}")
else:
    print("❌ 请完成 simulate_confounding_dag 函数！")

---

## ⚠️ Part 6: 碰撞偏差的危险

这是因果推断中最容易踩的坑之一！

### 好莱坞悖论 🎬

如果你只看好莱坞明星，你会发现：
- 颜值高的演技往往一般
- 演技好的长相往往一般

难道演技和颜值是负相关的？**不是的！**

这是因为「成为明星」是一个**碰撞变量**：

```
演技 → 成为明星 ← 颜值
```

当我们只看明星（条件于 C=1）时，打开了演技和颜值之间的虚假关联！

In [ ]:
def simulate_collider_bias(n: int = 2000, seed: int = 42):
    """
    模拟碰撞偏差
    
    DAG: T → C ← Y（T 和 Y 本来是独立的！）
    
    场景: 好莱坞悖论
    - T = 演技（0到10分）
    - Y = 颜值（0到10分）
    - C = 是否成为明星（需要演技+颜值 > 12）
    
    演技和颜值是独立的！但在明星群体中会呈现负相关。
    """
    np.random.seed(seed)
    
    # TODO: 生成演技 T（独立）
    T = None  # 👈 你的代码: np.random.uniform(0, 10, n)
    
    # TODO: 生成颜值 Y（独立）
    Y = None  # 👈 你的代码: np.random.uniform(0, 10, n)
    
    # TODO: 生成碰撞变量 C（是否成为明星）
    # 当 T + Y > 12 时成为明星
    C = None  # 👈 你的代码: (T + Y > 12).astype(int)
    
    df = pd.DataFrame({'演技': T, '颜值': Y, '明星': C})
    
    # 计算相关性
    overall_corr = np.corrcoef(T, Y)[0, 1]
    
    # 只看明星的相关性
    stars = df[df['明星'] == 1]
    conditional_corr = np.corrcoef(stars['演技'], stars['颜值'])[0, 1] if len(stars) > 10 else 0
    
    return df, overall_corr, conditional_corr

In [ ]:
# 运行碰撞偏差模拟
df_collider, overall, conditional = simulate_collider_bias(n=5000)

if overall is not None:
    print("🎬 好莱坞悖论模拟:")
    print("=" * 50)
    print(f"总人口中演技和颜值的相关性: {overall:.4f}")
    print(f"   解读: {'几乎不相关 ✅' if abs(overall) < 0.1 else '有相关性'}")
    print(f"\n只看明星时演技和颜值的相关性: {conditional:.4f}")
    print(f"   解读: {'强负相关 ⚠️' if conditional < -0.3 else '有负相关'}")
    
    print(f"\n明星占总人口: {df_collider['明星'].mean()*100:.1f}%")
else:
    print("❌ 请完成 simulate_collider_bias 函数！")

In [ ]:
# 可视化碰撞偏差
if overall is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 总人口
    ax1 = axes[0]
    stars = df_collider[df_collider['明星'] == 1]
    non_stars = df_collider[df_collider['明星'] == 0]
    
    ax1.scatter(non_stars['演技'], non_stars['颜值'], 
                alpha=0.3, c='gray', label='普通人', s=20)
    ax1.scatter(stars['演技'], stars['颜值'], 
                alpha=0.7, c='gold', label='明星', s=50, edgecolors='black')
    ax1.set_xlabel('演技')
    ax1.set_ylabel('颜值')
    ax1.set_title(f'总人口\n相关性 = {overall:.3f}')
    ax1.legend()
    
    # 图2: 只看明星
    ax2 = axes[1]
    ax2.scatter(stars['演技'], stars['颜值'], 
                alpha=0.7, c='gold', s=50, edgecolors='black')
    
    # 添加趋势线
    z = np.polyfit(stars['演技'], stars['颜值'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(stars['演技'].min(), stars['演技'].max(), 100)
    ax2.plot(x_line, p(x_line), 'r--', linewidth=2, label='趋势线')
    
    ax2.set_xlabel('演技')
    ax2.set_ylabel('颜值')
    ax2.set_title(f'只看明星群体\n相关性 = {conditional:.3f} (虚假负相关！)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 启示: 控制碰撞变量会创造虚假关联！")
    print("   在因果推断中，绝对不能控制碰撞变量。")

---

## 📝 Part 7: 思考题

### 问题 1: DAG 分析

给定以下 DAG:

```
X → T → Y ← U → X
```

回答:
- T 和 Y 之间有哪些路径？
- 哪些是后门路径？
- 最小调整集是什么？

**你的答案:**

*（在这里写下你的分析...）*

---

### 问题 2: 为什么控制碰撞变量会引入偏差？

**提示:** 想想好莱坞悖论的例子...

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 中介变量应该控制吗？

在以下 DAG 中：T → M → Y

- 什么时候应该控制 M？
- 什么时候不应该控制 M？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 未观测混淆

如果存在未观测的混淆变量 U（我们没有数据），有什么方法可以识别因果效应？

**提示:** 想想工具变量、断点回归、双重差分...

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎉 总结

### 核心知识点

| 概念 | 定义 | 规则 |
|-----|------|------|
| 混淆变量 | T ← X → Y | 必须控制 |
| 中介变量 | T → M → Y | 估计总效应时不控制 |
| 碰撞变量 | T → C ← Y | 绝对不能控制 |

### 后门准则

要无偏估计 T 对 Y 的效应：
1. 找出所有后门路径
2. 选择一组变量阻断所有后门路径
3. 确保这组变量不包含 T 的后代

### 下一步

现在我们知道了「应该控制什么」，下一个练习我们将深入学习**混淆偏差**——它有多大？怎么量化？怎么避免？

---

**「画图先行，分析后行」是因果推断的第一原则。**